In [49]:
import torch
from torch import nn
from torch.nn import functional as F
import torch.optim as optim

import pytorch_lightning as pl
from pytorch_lightning.accelerators import accelerator
from torchmetrics import functional as FM
from torchinfo import summary

from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [5, 3.5]
plt.rcParams["font.size"] = "8"

from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

def dataLoader(batch_size=128):
    train_dataset = MNIST('', transform=transforms.ToTensor(), train=True, download=True) ## 한 번 인터넷으로 가져온걸 매번 가져올 필요가 없기 때문에 가져올때 download True 로 하면 다음 부터는 다운로드 된 데이터를 사용한다.
    test_dataset = MNIST('', transform=transforms.ToTensor(), train=False, download=True)
    trainDataLoader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valDataLoader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return (trainDataLoader,valDataLoader)

trainDataLoader,valDataLoader = dataLoader()

In [3]:
loss_function = nn.CrossEntropyLoss() ## 회귀가 아닌 분류의 문제이므로 cross entropy 를 loss 로 사용한다. 

class mymodel( pl.LightningModule ):  
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(  
            nn.Flatten(),
            nn.Linear( 28*28, 10 ),
        )

    def forward(self, x):
        out = self.layers(x)
        return out
         
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = loss_function(y_pred, y)
        return loss
        
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)


# 모델 리뷰 

바로 전번에 돌린 모델을 보면, 이상한 점이 두 개 있다. 전에 사용했던 keras의 경우 categorical data의 cross entropy 를 계산하기 위해서는 output 되는 값을 확률 분포로 바꿔주고 (logit -> softmax 를 통한 prob), 또한 target y 값도 one-hot encoding 을 통해서 같은 dimmension 으로 바꿔줘야 확률분포의 차이 즉 크로스 엔트로피가 계산이 되었다. 근데 여기서는 그냥 y_pred (선형결합으로 이루어진 ~ logit 값, dim=10)  과  y (target 값, dim=1) 를 그냥 넣어주었다. 

이게 가능한 이유는 파이토치에서 이 부분을 자동으로 해주기 때문이다. 다음을 보자

In [46]:
tensor1 = torch.tensor( [[0, 0.5, 0.5, 0]] )
tensor2 = torch.tensor( [[0, 1, 0., 0]] )

out = loss_function(tensor1, tensor2)
out.item()

1.1672241687774658

이걸 softmax를 취하지 않은 형태의 tensor에 대해서 적용해도 별 문제 없다.

In [60]:
tensor1 = torch.tensor( [[0, 111.5, 111.5, 0]] )
tensor2 = torch.tensor( [[0, 1, 0., 0]] )

softmax_layer = nn.Softmax(dim=1)
tensor_softmaxed = softmax_layer(tensor1)  ## [[0, 0.5, 0.5, 0]] 으로 변환됨 

out = loss_function(tensor_softmaxed, tensor2)
out.item()

1.1672241687774658

물론 정확히 Softmax를 취한 값과는 값이 살짝 다른데 이유는 cross entropy 에서 실행되는 softmax 는 log_softmax 이기 때문이다. (하지만 어차피 loss 는 경향을 보는 거지 절대값 자체가 중요한건 아니기 때문에..)

In [62]:
tensor1 = torch.tensor( [[0, 111.5, 111.5, 0]] )
tensor2 = torch.tensor( [[0, 1, 0., 0]] )

out = loss_function(tensor1, tensor2)
out.item()

0.6931471824645996

In [ ]:
tensor1 = torch.tensor( [[0, 111.5, 111.5, 0]] )
tensor2 = torch.tensor( [[0, 1, 0., 0]] )

softmax_layer = nn.LogSoftmax(dim=1)
tensor_softmaxed = softmax_layer(tensor1)  ## [[0, 0.5, 0.5, 0]] 으로 변환됨 

out = loss_function(tensor_softmaxed, tensor2)
out.item()

그런데 더 놀라운 사실은 이 뿐만 아니라 one-hot encoding도 알아서 해준다. (true_y 가 pred_y 와 같은 shape 이면 그대로 확률을 비교하고, 그렇지 않다면 one-hot encoding을 진행한다)

In [70]:
tensor1 = torch.tensor( [[0, 111.5, 111.5, 0]] )
tensor2 = torch.tensor( [1] )  ## 확률 벡터가 아니라, 그냥 index만 넣어줘도! 

out = loss_function(tensor1, tensor2)
out.item()

0.6931471824645996

참 편리하다. 

## 모댈 개선

모델을 돌려서 학습까지 했지만, 뭐 나온게 없으므로 현 수준은 그냥 에러가 나지 않고 뭔가가 돌아갔다는 데 위안을 둬야한다. 이제 실제 성능을 확인하기 위해 몇가지를 개선하자

In [ ]:

class mymodel2( pl.LightningModule ):  
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(  
            nn.Flatten(),
            nn.Linear( 28*28, 10 ),
        )

    def forward(self, x):
        out = self.layers(x)
        return out
         
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = loss_function(y_pred, y)
        acc = FM.accuracy(y_pred, y, task="multiclass",num_classes=10)
        return loss
        
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)
